In [138]:
# -*- coding: utf-8 -*-

import os
import json
import nltk
import gensim
import numpy as np
from gensim import corpora, models, similarities
import pickle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.recurrent import LSTM,SimpleRNN
from keras import optimizers

In [139]:
#model=gensim.models.Word2Vec.load("chat_model1.w2v")
model =  gensim.models.KeyedVectors.load("chat_model1-300.w2v")

In [142]:
file=open('conv.json');
data = json.load(file)
cor=data["conversations"];

x=[]
y=[]

path2="corpus";

for i in range(len(cor)):
    for j in range(len(cor[i])):
        if j<len(cor[i])-1:
            x.append(cor[i][j]);
            y.append(cor[i][j+1]);

tok_x=[]
tok_y=[]
for i in range(len(x)):
    tok_x.append(nltk.word_tokenize(x[i].lower()))
    tok_y.append(nltk.word_tokenize(y[i].lower()))
    
    
sentend=np.ones((300,),dtype=np.float32) 

In [143]:
vec_x=[]
for sent in tok_x:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_x.append(sentvec)
    
vec_y=[]
for sent in tok_y:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_y.append(sentvec)           
    
    
for tok_sent in vec_x:
    tok_sent[14:]=[]
    tok_sent.append(sentend)
    

for tok_sent in vec_x:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend)    
            
for tok_sent in vec_y:
    tok_sent[14:]=[]
    tok_sent.append(sentend)
    

for tok_sent in vec_y:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend)             


C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [137]:
tok_y

[['i', 'am', 'doing', 'well', ',', 'how', 'about', 'you', '?'],
 ['i', "'m", 'also', 'good', '.'],
 ['that', "'s", 'good', 'to', 'hear', '.'],
 ['yes', 'it', 'is', '.'],
 ['hi'],
 ['how', 'are', 'you', 'doing', '?'],
 ['i', 'am', 'doing', 'well', '.'],
 ['that', 'is', 'good', 'to', 'hear'],
 ['yes', 'it', 'is', '.'],
 ['can', 'i', 'help', 'you', 'with', 'anything', '?'],
 ['yes', ',', 'i', 'have', 'a', 'question', '.'],
 ['what', 'is', 'your', 'question', '?'],
 ['could', 'i', 'borrow', 'a', 'cup', 'of', 'sugar', '?'],
 ['i', "'m", 'sorry', ',', 'but', 'i', 'do', "n't", 'have', 'any', '.'],
 ['thank', 'you', 'anyway'],
 ['no', 'problem'],
 ['i', 'am', 'doing', 'well', ',', 'how', 'about', 'you', '?'],
 ['i', 'am', 'also', 'good', '.'],
 ['that', "'s", 'good', '.'],
 ['what', 'good', 'news', '?'],
 ['i', 'ca', "n't", 'read', '.'],
 ['so', 'what', "'s", 'your', 'favorite', 'color', '?'],
 ['blue'],
 ['who',
  '?',
  'who',
  'is',
  'but',
  'a',
  'form',
  'following',
  'the',
  'func

In [144]:
vec_x=np.array(vec_x)
vec_y=np.array(vec_y)

In [146]:
vec_x[0].shape

(15, 300)

In [147]:
vec_x
#Jumlah kalimat, panjang array, fitur vector
#(86,15,300)

array([[[-2.21359670e-01,  1.36077181e-01, -1.32402852e-01, ...,
         -3.16020548e-01,  6.79968894e-02,  6.09610528e-02],
        [-1.23945788e-01,  8.81088376e-02, -9.22681838e-02, ...,
         -1.43739864e-01,  4.36660573e-02,  4.92373947e-04],
        [-1.73033252e-01,  1.32049993e-01, -1.24457426e-01, ...,
         -2.80463159e-01,  7.57281259e-02,  3.90171781e-02],
        ...,
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
          1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
          1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
          1.00000000e+00,  1.00000000e+00,  1.00000000e+00]],

       [[-5.15709743e-02,  4.58154716e-02, -4.50045094e-02, ...,
         -2.73952782e-01,  9.89845619e-02,  1.91080216e-02],
        [-1.24328457e-01,  7.30226710e-02, -6.96329251e-02, ...,
         -2.24732980e-01,  4.28869203e

In [126]:
x_train,x_test, y_train,y_test = train_test_split(vec_x, vec_y, test_size=0.2, random_state=1)
    
model=Sequential()
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='cosine_proximity', optimizer=adam, metrics=['accuracy'])

# Train here

In [119]:
model.fit(x_train, y_train, epochs=500,validation_data=(x_test, y_test))
model.save('LSTM500.h5');

Train on 68 samples, validate on 18 samples
Epoch 1/500
68/68 [==============================] - 3s 38ms/step - loss: -0.7249 - acc: 0.0745 - val_loss: -0.7216 - val_acc: 0.2296
Epoch 2/500
68/68 [==============================] - 0s 7ms/step - loss: -0.7464 - acc: 0.2245 - val_loss: -0.7328 - val_acc: 0.2852
Epoch 3/500
68/68 [==============================] - 0s 6ms/step - loss: -0.7541 - acc: 0.2539 - val_loss: -0.7402 - val_acc: 0.2852
Epoch 4/500
68/68 [==============================] - 0s 6ms/step - loss: -0.7616 - acc: 0.2539 - val_loss: -0.7470 - val_acc: 0.2852
Epoch 5/500
68/68 [==============================] - 0s 6ms/step - loss: -0.7682 - acc: 0.2539 - val_loss: -0.7529 - val_acc: 0.2852
Epoch 6/500
68/68 [==============================] - 0s 6ms/step - loss: -0.7739 - acc: 0.2539 - val_loss: -0.7582 - val_acc: 0.2852
Epoch 7/500
68/68 [==============================] - 0s 6ms/step - loss: -0.7795 - acc: 0.2539 - val_loss: -0.7644 - val_acc: 0.2852
Epoch 8/500
68/68 [=====

Epoch 62/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8481 - acc: 0.2510 - val_loss: -0.8363 - val_acc: 0.2741
Epoch 63/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8483 - acc: 0.2510 - val_loss: -0.8357 - val_acc: 0.2741
Epoch 64/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8482 - acc: 0.2510 - val_loss: -0.8373 - val_acc: 0.2741
Epoch 65/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8493 - acc: 0.2529 - val_loss: -0.8384 - val_acc: 0.2778
Epoch 66/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8488 - acc: 0.2539 - val_loss: -0.8382 - val_acc: 0.2741
Epoch 67/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8493 - acc: 0.2510 - val_loss: -0.8350 - val_acc: 0.2704
Epoch 68/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8480 - acc: 0.2451 - val_loss: -0.8348 - val_acc: 0.2704
Epoch 69/500
68/68 [==============================] - 0s 6ms/s

Epoch 123/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8496 - acc: 0.2529 - val_loss: -0.8382 - val_acc: 0.2815
Epoch 124/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8504 - acc: 0.2539 - val_loss: -0.8375 - val_acc: 0.2741
Epoch 125/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8503 - acc: 0.2529 - val_loss: -0.8353 - val_acc: 0.2741
Epoch 126/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8493 - acc: 0.2441 - val_loss: -0.8339 - val_acc: 0.2704
Epoch 127/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8487 - acc: 0.2431 - val_loss: -0.8354 - val_acc: 0.2704
Epoch 128/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8495 - acc: 0.2451 - val_loss: -0.8369 - val_acc: 0.2741
Epoch 129/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8502 - acc: 0.2539 - val_loss: -0.8384 - val_acc: 0.2852
Epoch 130/500
68/68 [==============================] - 

Epoch 184/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8552 - acc: 0.2529 - val_loss: -0.8366 - val_acc: 0.2852
Epoch 185/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8560 - acc: 0.2539 - val_loss: -0.8345 - val_acc: 0.2815
Epoch 186/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8568 - acc: 0.2539 - val_loss: -0.8365 - val_acc: 0.2852
Epoch 187/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8580 - acc: 0.2539 - val_loss: -0.8372 - val_acc: 0.2852
Epoch 188/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8575 - acc: 0.2539 - val_loss: -0.8369 - val_acc: 0.2852
Epoch 189/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8575 - acc: 0.2539 - val_loss: -0.8356 - val_acc: 0.2741
Epoch 190/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8573 - acc: 0.2500 - val_loss: -0.8344 - val_acc: 0.2704
Epoch 191/500
68/68 [==============================] - 

Epoch 245/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8494 - acc: 0.2392 - val_loss: -0.8381 - val_acc: 0.2741
Epoch 246/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8492 - acc: 0.2529 - val_loss: -0.8375 - val_acc: 0.2815
Epoch 247/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8504 - acc: 0.2539 - val_loss: -0.8353 - val_acc: 0.2778
Epoch 248/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8501 - acc: 0.2539 - val_loss: -0.8355 - val_acc: 0.2852
Epoch 249/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8503 - acc: 0.2539 - val_loss: -0.8352 - val_acc: 0.2815
Epoch 250/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8500 - acc: 0.2539 - val_loss: -0.8336 - val_acc: 0.2741
Epoch 251/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8497 - acc: 0.2529 - val_loss: -0.8366 - val_acc: 0.2852
Epoch 252/500
68/68 [==============================] - 

Epoch 306/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8517 - acc: 0.2510 - val_loss: -0.8399 - val_acc: 0.2778
Epoch 307/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8515 - acc: 0.2431 - val_loss: -0.8390 - val_acc: 0.2704
Epoch 308/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8519 - acc: 0.2373 - val_loss: -0.8378 - val_acc: 0.2556
Epoch 309/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8514 - acc: 0.2392 - val_loss: -0.8388 - val_acc: 0.2667
Epoch 310/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8522 - acc: 0.2382 - val_loss: -0.8380 - val_acc: 0.2630
Epoch 311/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8525 - acc: 0.2382 - val_loss: -0.8381 - val_acc: 0.2630
Epoch 312/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8527 - acc: 0.2392 - val_loss: -0.8379 - val_acc: 0.2667
Epoch 313/500
68/68 [==============================] - 

Epoch 367/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8609 - acc: 0.2539 - val_loss: -0.8355 - val_acc: 0.2852
Epoch 368/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8609 - acc: 0.2539 - val_loss: -0.8349 - val_acc: 0.2852
Epoch 369/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8613 - acc: 0.2539 - val_loss: -0.8352 - val_acc: 0.2815
Epoch 370/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8613 - acc: 0.2539 - val_loss: -0.8341 - val_acc: 0.2778
Epoch 371/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8621 - acc: 0.2539 - val_loss: -0.8303 - val_acc: 0.2741
Epoch 372/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8617 - acc: 0.2500 - val_loss: -0.8299 - val_acc: 0.2741
Epoch 373/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8623 - acc: 0.2510 - val_loss: -0.8318 - val_acc: 0.2741
Epoch 374/500
68/68 [==============================] - 

Epoch 428/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8656 - acc: 0.2539 - val_loss: -0.8327 - val_acc: 0.2741
Epoch 429/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8658 - acc: 0.2520 - val_loss: -0.8314 - val_acc: 0.2741
Epoch 430/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8659 - acc: 0.2490 - val_loss: -0.8303 - val_acc: 0.2704
Epoch 431/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8659 - acc: 0.2490 - val_loss: -0.8299 - val_acc: 0.2741
Epoch 432/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8660 - acc: 0.2510 - val_loss: -0.8305 - val_acc: 0.2741
Epoch 433/500
68/68 [==============================] - 0s 6ms/step - loss: -0.8645 - acc: 0.2529 - val_loss: -0.8304 - val_acc: 0.2815
Epoch 434/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8667 - acc: 0.2539 - val_loss: -0.8281 - val_acc: 0.2852
Epoch 435/500
68/68 [==============================] - 

Epoch 489/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8672 - acc: 0.2539 - val_loss: -0.8297 - val_acc: 0.2778
Epoch 490/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8674 - acc: 0.2539 - val_loss: -0.8265 - val_acc: 0.2778
Epoch 491/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8693 - acc: 0.2539 - val_loss: -0.8260 - val_acc: 0.2852
Epoch 492/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8667 - acc: 0.2539 - val_loss: -0.8270 - val_acc: 0.2852
Epoch 493/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8658 - acc: 0.2539 - val_loss: -0.8292 - val_acc: 0.2852
Epoch 494/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8699 - acc: 0.2539 - val_loss: -0.8291 - val_acc: 0.2852
Epoch 495/500
68/68 [==============================] - 0s 7ms/step - loss: -0.8673 - acc: 0.2539 - val_loss: -0.8310 - val_acc: 0.2852
Epoch 496/500
68/68 [==============================] - 

# Try Predict

In [125]:
predictions=model.predict(x_test) 
mod = gensim.models.Word2Vec.load('chat_model1-300.w2v');   
[mod.most_similar([predictions[10][i]])[0] for i in range(15)]

C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('Meleleh', 0.22824956476688385),
 ('Meleleh', 0.2318115234375),
 ('Meleleh', 0.2318737953901291),
 ('Meleleh', 0.23178881406784058),
 ('Meleleh', 0.23159852623939514),
 ('Meleleh', 0.23136582970619202),
 ('Meleleh', 0.2311236709356308),
 ('Meleleh', 0.23089399933815002),
 ('Meleleh', 0.2306804358959198),
 ('Meleleh', 0.23049503564834595),
 ('Meleleh', 0.23033496737480164),
 ('Meleleh', 0.23019729554653168),
 ('Meleleh', 0.23007968068122864),
 ('Meleleh', 0.22997963428497314),
 ('Meleleh', 0.2298947274684906)]

In [124]:
x_test

array([[[-0.1150071 ,  0.12650718, -0.11326685, ..., -0.19074586,
          0.07033682,  0.01983952],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        ...,
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ]],

       [[-0.21620193,  0.13295627, -0.17647928, ..., -0.33258867,
          0.12898433,  0.05734357],
        [-0.23041238,  0.17280413, -0.1521971 , ..., -0.3402636 ,
          0.15490232,  0.05494078],
        [-0.17749351,  0.29426488, -0.27981547, ..., -0.22992593,
          0.2164393 ,  0.00579983],
        ...,
        [ 1.        ,  1.        ,  1.        , ...,  